In [3]:
#Creating DB engine(Sqlite), Ingestion of dataframe into DB(Sql Engine), to read files from particular directory, Setting up logging handler and logging of data
import pandas as pd
import os
from sqlalchemy import create_engine
import logging
import time
import sqlite3

for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(filename='/content/ingestiondb.log', level= logging.DEBUG,format='%(asctime)s -%(levelname)s -%(message)s',filemode='a')
logging.debug("Logging setup complete.")

engine=create_engine('sqlite:///inventory.db')


def ingest_db(df,table_name,engine):
  '''This function ingests data frames/data to DB server(EngineName) continuously'''
  df.to_sql(table_name, con=engine, if_exists='replace', index=False)#Coverting/Ingesting dataframe to sql engine(Tablename, engine,incase of continuous

'''This function will load the CSVs as dataframe and ingest into db'''#data use append instead of if_exists,Index=F do not want indexing)
def load_raw_data():
  '''This function loads csv's as dataframes and ingest into DB'''
  start= time.time()
  for fileindir in os.listdir('/content/'): #To read files from particular directory os.listdir(path of directory)
    if '.csv' in fileindir:                             #To categorize filenames which has .csv at the end
      df=pd.read_csv('/content/'+fileindir) #To read csv files (path + filename)
      print(df.shape)
      logging.info(f'Ingesting {fileindir} in db')
      ingest_db(df,fileindir[:-4],engine)              #Calling Ingestion function(using slicing to remove .csv from the end of filename to create tablename)
      end=time.time()
      total_time=(end-start)/60
      logging.info(f'------Ingestion complete for {fileindir} in db-----')
      logging.info(f'\nTotal time taken is {total_time} minutes')

if __name__=='__main__':
  load_raw_data()

# connect to your database
conn=sqlite3.connect('/content/inventory.db')  #Establishing DB connection
# list all tables
tables =pd.read_sql_query('SELECT name FROM sqlite_master WHERE type="table"',conn) #Checking tables in database
tables
pd.read_sql('select count(*) from purchases', conn) #To find specific count of records in the table
for table in tables['name']:  #To print count of records in each table
  print('-'*50,f'{table}','-'*50)
  print('count of records:',pd.read_sql(f"select count (*) as count from {table}",conn)['count'].values[0])
  display(pd.read_sql(f"select * from {table} limit 5",conn))
#conn.close()
